In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter

#we actually need to concatenate a bunch of files to get who annotated what  
stem = "/home/blitt/projects/podcasts/annotation/hostGuestAnnotation/annotation_output/"
folderNames  = ["5a4d8a8f0b817800013032ea", "5ece9e3b3a19db0cf5af6692", "6022ad31accb29025f079d1c", "65537e115e58fe12b3fcf2f2", "6615cd319d6d5ee3879758ee", "5e318768c7b20d000d0830ce", "5ff66092dfb1e6211decef3e", "6440d887ac943c907678ec9f", "65e739f565a8c818b009f83b"]
dfList = []

for fName in folderNames: 
    fPath = stem + fName + "/annotated_instances.jsonl" 
    currDf = pd.read_json(fPath, orient="records", lines=True)
    currDf["annotatorId"] = fName
    dfList.append(currDf)
df = pd.concat(dfList)

dictList = list(df["label_annotations"])
labels = [int(list(item["sentiment"].values())[0]) for item in dictList]
df["annotation"] = labels

In [4]:
df.head() 

,id,displayed_text,label_annotations,span_annotations,behavioral_data,annotatorId,annotation
0,https://feeds.soundcloud.com/stream/825188326-...,<span style='background-color:#00FF00'>Target ...,{'sentiment': {'neither': '3'}},{},{'time_string': 'Time spent: 0d 0h 0m 12s '},5a4d8a8f0b817800013032ea,3
1,https://www.buzzsprout.com/998860/3752489-corn...,<span style='background-color:#00FF00'>Target ...,{'sentiment': {'host': '1'}},{},{'time_string': 'Time spent: 0d 0h 0m 18s '},5a4d8a8f0b817800013032ea,1
2,https://anchor.fm/s/11028184/podcast/play/1421...,<span style='background-color:#00FF00'>Target ...,{'sentiment': {'guest': '2'}},{},{'time_string': 'Time spent: 0d 0h 0m 17s '},5a4d8a8f0b817800013032ea,2
3,https://api.spreaker.com/download/episode/2686...,<span style='background-color:#00FF00'>Target ...,{'sentiment': {'neither': '3'}},{},{'time_string': 'Time spent: 0d 0h 0m 16s '},5a4d8a8f0b817800013032ea,3
4,https://feeds.soundcloud.com/stream/829978630-...,<span style='background-color:#00FF00'>Target ...,{'sentiment': {'guest': '2'}},{},{'time_string': 'Time spent: 0d 0h 0m 12s '},5a4d8a8f0b817800013032ea,2


In [40]:
#find high disagreement examples
instGrouped = df[["id", "annotatorId", "annotation", "displayed_text"]].groupby("id").agg(list)
instGrouped["displayed_text"] = instGrouped["displayed_text"].apply(lambda x: x[0])

In [41]:

def getNonModal(inRow): 
    counts = Counter(inRow["annotation"])
    modalVal, modeCount = counts.most_common(1)[0]

    return [modalVal, modeCount]

instGrouped[["modalVal", "modalCount"]] = instGrouped.apply(getNonModal, axis=1).to_list()

In [43]:
def pPrint(inText): 
    tList = inText.split()
    lineLen = 20
    currText = ""
    for count, item in enumerate(tList): 
        if count % lineLen == 0: 
            print(currText)
            currText = ""
        currText += item + " "

In [45]:
#totalDisDf = df.set_index("id", drop=False).loc[totalDisList].drop_duplicates("id") 
totalDisDf = instGrouped[instGrouped["modalCount"] == 1]
for i, row in totalDisDf.iterrows(): 
    print(row["annotation"])
    pPrint(row["displayed_text"])

[1, 3, 2]

<span style='background-color:#00FF00'>Target Entity: Jack Buckling</span><br><br><strong>Podcast Description:</strong><br>My first podcast! subject bees.<br><br><strong>Podcast Episode Description:</strong><br> This is my second podcast. [hyperlink] Link to 
mentioned website.. <br><br><strong>Podcast Transcript Excerpt:</strong><br>...which was on bees. Now if you were watching last week you will know that I 
told you about the declining population of bees and how actually important they are to humans. But today we will 
be reviewing what I briefly talked about last week which was about pesticides. Now pesticides we've all used them they 
help with getting rid of plants that we don't really want in our garden and they are sometimes quite useful 
but there is a downfall to that convenience. Some pesticides can be very very harmful to insects. Today we're going 
to be overviewing that and seeing what you can do to one stop using pesticides and find alternatives and also 
to 

In [46]:
totalDisDf = instGrouped[instGrouped["modalCount"] == 2]
for i, row in totalDisDf.iterrows(): 
    print(row["annotation"])
    pPrint(row["displayed_text"])

[1, 2, 2]

<span style='background-color:#00FF00'>Target Entity: Josh Paskel</span><br><br><strong>Podcast Description:</strong><br>The NCAA Podcasts channel uses multiple different shows to interview individuals who have unique insight 
into the world of college sports. Produced by the NCAA, episodes delve into the most pressing topics of the day 
and offer behind-the-scenes points of view from those who shape college athletics.<br><br><strong>Podcast Episode Description:</strong><br>Andy Katz and NCAA Chief Medical Officer 
Dr. Brian Hainline are joined by Kansas State University volleyball player Brynn Carlson and University of Kentucky football player <span 
style="background-color:#00FF00">Josh</span> Paschal. Brynn and <span style="background-color:#00FF00">Josh</span> provide their perspectives on returning to campus with COVID-19 protections and procedures in place.<br><br><strong>Podcast 
Transcript Excerpt:</strong><br>(upbeat music) - Welcome everyone to our latest edition o

In [ ]:
"""
NOTES ON 2/3 AGREEMENT CASES: 
on google slide deck 
"""